# Lab 4, Exercise 4

In [4]:
import numpy as np
import pandas as pd

## Load data 


In [36]:
# Load the data in the following two CSVs:
# data/exercise4/lab4_normal_data.csv
# data/exercise4/lab4_malicious_data.csv
# The first consists completely of normal data, while the second consists completely of malicious data
# Note: Both sets of data contain the same features used in Exercise 1; the data has already been preprocessed
# (i.e., you can keep all the features and there are no labels in the CSVs)

# CODE HERE
norm_data = pd.read_csv('data/exercise4/lab4_normal_data.csv')
mal_data = pd.read_csv('data/exercise4/lab4_malicious_data.csv')

,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,112,674,5100,914280,102.373797,254,252,5.273881e+03,952453.437500,2,...,1,1,1,1,0,0,1,2,1,0
1,2,0,92,0,142857.140900,254,0,5.257143e+07,0.000000,0,...,2,1,1,1,0,0,0,2,5,0
2,48,230,2440,308245,74.802232,254,252,5.163244e+03,663023.250000,2,...,1,1,1,1,0,0,1,3,1,0
3,66,370,3202,482333,82.376719,254,252,4.778229e+03,728748.000000,2,...,1,1,1,1,0,0,1,1,2,0
4,112,694,5074,915107,104.469111,254,252,5.221120e+03,948697.875000,2,...,1,1,1,1,0,0,1,1,1,0
5,10,8,1308,354,24.973080,254,252,1.384390e+04,3643.131592,2,...,1,1,1,2,0,0,0,1,2,0
6,2,0,92,0,111111.107200,254,0,4.088889e+07,0.000000,0,...,1,1,1,1,0,0,0,1,1,0
7,10,6,932,268,61.824057,254,252,2.766421e+04,7385.914062,2,...,1,1,1,2,0,0,0,1,2,0
8,66,362,3216,481994,80.973792,254,252,4.806088e+03,729201.062500,2,...,1,1,1,1,0,0,1,1,2,0
9,2,0,92,0,76923.077790,254,0,2.830769e+07,0.000000,0,...,1,1,1,1,0,0,0,2,1,0


In [35]:
# Create 15 datasets, where the ith dataset consists of:
# - all normal data
# - only the ith malicious datapoint

# CODE HERE
datasets = []
for i in range(mal_data.shape[0]):
    datasets.append(pd.concat([norm_data,mal_data.iloc[i:i+1,:]],ignore_index=True))

## Anomaly detection

In [51]:
# For each dataset, run isolation forests
#
# Use the following evaluation metric:
# - rank the anomalousness of each datapoint using the isolation forest
# - record the list index of each attack datapoint when sorting from most to least unusual
#     - e.g., if the attack datapoint is at index 0 in the list, we want to record the value 0
#
# Note: don't worry about ties in ranking
# Hint: What is the difference between isolation forest's 'decision_function' and 'predict' methods? 

# CODE HERE
from sklearn.ensemble import IsolationForest
list_idxs = []
for i in range(len(datasets)):
    model = IsolationForest(behaviour='new', contamination=0.1, random_state=42).fit(datasets[i])
    anom_scores = model.decision_function(datasets[i])
    anom_list = [x for x in zip(range(len(anom_scores)), anom_scores)]
    anom_list.sort(key=lambda anom: anom[1])
    list_idxs.append([x[0] for x in anom_list].index(datasets[i].shape[0]-1))
print('List index of each attack datapoint:\n{}'.format(list_idxs))

List index of each attack datapoint:
[161, 18579, 1562, 1412, 0, 28441, 15273, 25003, 722, 17906, 139, 364, 364, 1130, 172]


## Questions:
1) Why is there no separate training and test set?

There is no separate training and test set because this is not a problem using a supervised model, such as regression or classification. This problem utilizes an unsupervised model to "cluster" datapoints and find anomalies in the datapoints, which does not require a training and test split.

2) What is the metric measuring?  What would be a perfect score?  Bonus: What is the expected performance of an outlier detector that assigns a random score to each datapoint?

The anomaly score is measuring the degree to which a datapoint is an anomaly, while the index metric measures how anomalous the datapoint is compared to all of the normal data. A perfect score for the index metric would be a 0, indicating that the datapoint is the most anomalous of all the datapoints.

3) How well does the isolation forest perform compared to a perfect score? Bonus: How well does the isolation forest perform compared to a random detector?

The isolation forest does not perform the best compared to a perfect score, despite getting it once, since there are lots of indexes that are very far off a perfect score.

4) What are some issues that would prevent this model from being practically deployed?

5) What might happen if we inject five attack datapoints at a time?  What might happen if we inject 100 attack datapoints at a time?

6) What is the effect of the parameters max_features and max_samples?  What other parameters could you adjust to change performance?

Optional: What are some alternative anomaly detection models one could use instead of an isolation forest? Bonus: Try one of these alternatives and compare performance.